In [1]:
!pip uninstall -y pacmap && pip install git+https://github.com/nfultz/PaCMAP
!pip install git+https://github.com/erichson/optht/
!pip uninstall -y eonacs && pip install  git+https://github.com/njnmco/smc #--log /dev/stderr

Found existing installation: pacmap 0.5.4
Uninstalling pacmap-0.5.4:
  Successfully uninstalled pacmap-0.5.4
  Cloning https://github.com/nfultz/PaCMAP to /tmp/pip-req-build-bbdwpmgr
  Running command git clone -q https://github.com/nfultz/PaCMAP /tmp/pip-req-build-bbdwpmgr
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pacmap: filename=pacmap-0.5.4-py3-none-any.whl size=15389 sha256=5b86fa83fa34324cd2f7b6f120b60f10021bf214a4d93ce383556a06d11fb2c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-i8gj5ygu/wheels/41/57/45/b6ddb0ae83722ba92f5c6de869a1b80673a2b80eb41c64dc0b
Successfully built pacmap
  Cloning https://github.com/erichson/optht/ to /tmp/pip-req-build-gi2zt8a1
  Running command git clone -q https://github.com/erichson/optht/ /tmp/pip-req-build-gi2zt8a1
Found existing installation: eonacs 1.0.0
Uninstalling eonacs-1.0.0:
  Successfully uninstalled eonacs-1.0.0
  Cloning https://

In [2]:
import eonacs.common.util as util

util.colab_map_drive()


Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:
# tasks = pd.read_sql("select distinct substr(onetsoc_code, 1, 2) as soc,  task as text from task_statements",   con)


In [5]:
import eonacs.common.reduction as reduction


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [6]:
# import eonacs.common.dbert as dbert
# import torch
# from transformers import AutoModel
# tokenizer, model = dbert.dbert()
# fine  = AutoModel.from_pretrained("/content/drive/MyDrive/smc/models/dbert_tasks/")
# hyper = AutoModel.from_pretrained("/content/drive/MyDrive/smc/models/dbert_tasks_hyper/")

In [7]:
# hyper_e = dbert.get_embeddings(tasks['text'], tokenizer, hyper)

In [8]:
# pd.to_pickle((tasks,hyper_e), "/content/drive/MyDrive/smc/data/tasks_dbert_hyper_e.pkl.gz") 

In [9]:
tasks, hyper_e = pd.read_pickle("/content/drive/MyDrive/smc/data/tasks_dbert_hyper_e.pkl.gz")

In [10]:
s = np.linalg.svd(hyper_e, compute_uv=False)

In [11]:
import optht
optht.optht(hyper_e, s)

259

In [12]:
from sklearn.preprocessing import StandardScaler
features = StandardScaler().fit_transform(hyper_e)

In [13]:
# %%time 
# m = reduction.PaCMAP2(120)
# m.fit(features)

In [14]:
# dim 2 => 16s
# 5 => 23s
# 20 => 41
# 50 => 61
# 120 => 97 (cpu 163)
# 260 => 103

In [25]:
def cv(features, nfolds, loss):
  folds = np.random.randint(0, nfolds, features.shape[0])
  def score_fold(i):
    train, test = features[folds != i, :], features[folds == i, :]
    m = reduction.PaCMAP2(10)
    pred = m.fit(train).transform(test)
    return loss(test, pred)
  return [score_fold(i) for i in range(nfolds)]

In [26]:
import scipy

def loss(Y, Yhat):
  test_dist = scipy.spatial.distance.pdist(Y)
  pred_dist = scipy.spatial.distance.pdist(Yhat)
  return 1 - scipy.stats.spearmanr(test_dist, pred_dist).correlation

In [27]:
%time losses = cv(features, 10, loss)
losses

CPU times: user 8min 47s, sys: 19 s, total: 9min 6s
Wall time: 7min 17s


[0.5729828478950075,
 0.5686685791751551,
 0.5684038268211868,
 0.5379708151078821,
 0.5580278309856297,
 0.5660943694899694,
 0.5487347798675333,
 0.5565656076775674,
 0.5380929078312922,
 0.5758165044381048]

In [28]:
sorted(losses)[1] # d = 10

0.5380929078312922

In [24]:
sorted(losses)[1] # d = 2

0.6131274957075751